In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:

DATASETS = {
    "big_mac": {
        "mid":   ("X_features_fed_funds_mid_big_mac.csv",   "y_target_fed_funds_mid_big_mac.csv"),
        "lower": ("X_features_fed_funds_lower_big_mac.csv", "y_target_fed_funds_lower_big_mac.csv"),
        "upper": ("X_features_fed_funds_upper_big_mac.csv", "y_target_fed_funds_upper_big_mac.csv"),
    },
    "no_big_mac": {
        "mid":   ("X_features_fed_funds_mid_no_big_mac.csv",   "y_target_fed_funds_mid_no_big_mac.csv"),
        "lower": ("X_features_fed_funds_lower_no_big_mac.csv", "y_target_fed_funds_lower_no_big_mac.csv"),
        "upper": ("X_features_fed_funds_upper_no_big_mac.csv", "y_target_fed_funds_upper_no_big_mac.csv"),
    }
}


# SELECT FEATURE SET
USE_BIG_MAC = True         # True = Big Mac engineered set, False = FRED-only
TARGET_VERSION = "mid"     # "mid", "lower", or "upper"

subset = "big_mac" if USE_BIG_MAC else "no_big_mac"

X_file, y_file = DATASETS[subset][TARGET_VERSION]

print("Using X file:", X_file)
print("Using y file:", y_file)


Using X file: X_features_fed_funds_mid_big_mac.csv
Using y file: y_target_fed_funds_mid_big_mac.csv


In [7]:
# Chronological split
split_index = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

NameError: name 'df' is not defined

In [ ]:
baseline_pred = np.full_like(y_test, y_train.mean(), dtype=float)
baseline_pred_persist = np.full_like(y_test, y_train.iloc[-1], dtype=float)

def evaluate(y_true, y_pred):
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "R2": r2_score(y_true, y_pred)
    }

print("Mean baseline:", evaluate(y_test, baseline_pred))
print("Persistence baseline:", evaluate(y_test, baseline_pred_persist))

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.01),
    "ElasticNet": ElasticNet(alpha=0.01, l1_ratio=0.5),
    "SVR (RBF)": SVR(kernel="rbf", C=10, gamma=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = evaluate(y_test, preds)

pd.DataFrame(results).T

plt.figure(figsize=(12,5))
plt.plot(y_test.values, label="True")

for name, model in models.items():
    pred = model.predict(X_test)
    plt.plot(pred, label=name)

plt.legend()
plt.title("Model Predictions vs True FFR")
plt.show()

rf = models["Random Forest"]
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values().plot(kind="barh", figsize=(10,6))
plt.title("Random Forest Feature Importance")
plt.show()

pd.DataFrame(results).T.to_csv("model_performance.csv")